# Diabetes Classification

In [2]:
from pandas import read_csv


df = read_csv('part1.csv')
df.head()

<bound method NDFrame.head of        Unnamed: 0     ID  DIABETE3  _RFHYPE5  TOLDHI2  _CHOLCHK   _BMI5  \
0               0      0       3.0         2      1.0         1  4018.0   
1               1      1       3.0         1      2.0         2  2509.0   
2               2      2       3.0         1      1.0         1  2204.0   
3               3      3       3.0         2      1.0         1  2819.0   
4               3      3       3.0         2      1.0         1  2819.0   
...           ...    ...       ...       ...      ...       ...     ...   
99995       90864  90864       3.0         1      1.0         1  2658.0   
99996       90865  90865       1.0         2      1.0         1  2637.0   
99997       90865  90865       1.0         2      1.0         1  2637.0   
99998       90866  90866       3.0         2      2.0         1  2246.0   
99999       90867  90867       3.0         1      NaN         3  2744.0   

       SMOKE100  CVDSTRK3  _MICHD  ...  HLTHPLN1  MEDCOST  GENHLTH  M